In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Non-std libs
import matplotlib.pyplot as plt

## Local modules
from scn_rrd import rrd_meta_utils, rrd_utils

In [ ]:
epc_locations = [
    'FCS',
    'nickelsville-cd',
    'northlake',
    'occ-oromo',
    'sps-franklin',
    'sps-garfield',
]
lte_locations = [
    'lihi-southend',
    'nickelsville-cd',
]

meta = rrd_meta_utils.read_meta()
meta = meta[
    (
        ( meta['location'].isin(epc_locations) ) &
        ( meta['device_group_name'].isin(['EPC', 'Backhaul gateway']) )
    ) |
    (
        ( meta['location'].isin(lte_locations) ) &
        ( meta['device_group_name'] == '' )
    )
]
deviceHostnames = meta['hostname'].unique()

In [ ]:
deviceHostname_to_df = rrd_utils.read_rrds(
    deviceHostnames,
    rrd_meta_utils.DEVICE_AGNOSTIC_RRD_FILENAMES.avail_weekly,
    '-2month',
)

In [ ]:
(fig, ax) = plt.subplots(figsize=(12,6))
for (deviceHostname, df) in deviceHostname_to_df.items():
    ax.plot(df['time'], df['availability'], label=deviceHostname)
ax.set_title('Availability')
ax.set_ylabel('Percentage')
ax.tick_params(axis='x', labelrotation=90)
ax.legend()